# Case study of uniaxial tensile tests

This is an example of how to use ParaMaterial to process a dataset of uniaxial tensile test measurements.



## Data preparation
- [ ] **Make the info table.** This should be a .csv or .xlsx file where the columns are metadata categories. The first column must contain the *test id* --- a unique string that links the information about the test in that row to the measurements data in a separate file.
- [ ] **Format the data files.** The time-series measurements for each test should be saved in a file named "*\<test id\>*.csv".
- [ ] **Exploratory data analysis.** Quick investigation and categorisation of subsets. Visualisation of data.
- [ ] **Screening.**


Import libraries.

In [1]:
import os
import shutil

from matplotlib import pyplot as plt

import paramaterial as pam

### Make the info table

In [3]:
backup_info = pam.preparing.make_info_table('data/00 backup data',
                                            ['test id', 'old filename', 'test type', 'material', 'temperature', 'rate'])
backup_info.head()

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'data/00 backup data'

Add the metadata and apply a naming convention in the "test id" column.

In [ ]:
for i, filename in enumerate(os.listdir('data/00 backup data')):
    test_id = f'testID_{i + 1:03d}'
    test_type, temperature, material = filename.split('_')[0:3]
    rate = 8.66e-4
    backup_info.loc[
        backup_info['old filename'] == filename,
        ['test id', 'test type', 'material', 'temperature', 'rate']
    ] = [test_id, test_type, material, temperature, rate]

backup_info.to_excel('info/00 backup info.xlsx', index=False)
backup_info.head()

Copy backup data and info into raw data and info, then leave backup alone in all further steps.

In [ ]:
old_dir, new_dir = 'data/00 backup data', 'data/01 raw data'
if os.path.isdir(new_dir):  # delete new_dir and files if they already exist
    shutil.rmtree(new_dir)
shutil.copytree(old_dir, new_dir)

shutil.copy('info/00 backup info.xlsx', 'info/01 raw info.xlsx')

### Preparing raw data
Rename files according to test id.

In [ ]:
pam.preparing.rename_by_test_id('data/01 raw data', 'info/01 raw info.xlsx')

Check that column headers of data files are all the same.

In [ ]:
pam.preparing.check_column_headers('data/01 raw data')

### Exploratory data analysis

Now that we have prepared the data, we can use the DataSet class to create a plot of all the raw data.

In [ ]:
from paramaterial.plug import DataSet

fig, ax = plt.subplots(1, 1, figsize=(9, 6))
raw_dataset = DataSet('data/01 raw data', 'info/01 raw info.xlsx')
raw_dataset.plot(ax=ax, colorby='temperature', x='Strain', y='Stress_MPa', xlabel='Strain', ylabel='Stress (MPa)', legend=False)

We can also plot subsets of the dataset with different styles. Here, we style according to test type.

In [ ]:
raw_dataset.plot(colorby='temperature', styleby='test type', x='Strain', y='Stress_MPa', ylabel='Stress (MPa)', legend=False, cbar=True, widthby='material', alpha=0.8, grid=True)

Alternatively, we might want to split the data into subplots.

In [ ]:
raw_dataset.subplots(2,2,colsby='material',col_keys=['A,B,'])